In [1]:
!curl https://raw.githubusercontent.com/mcruzr0609/carla-python-api/master/PythonAPI/carla/dist/carla-0.9.10-py3.7-linux-x86_64.egg > carla-0.9.10-py3.7-linux-x86_64.egg
 
import glob
import os
import sys
 
try:
    sys.path.append(glob.glob('carla-0.9.10-py3.7-linux-x86_64.egg')[0])
except IndexError:
    pass
 
    
from collections import deque
import math
import numpy as np
from enum import Enum
import random
import networkx as nx

import carla

from carla import ColorConverter as cc

import collections
import datetime
import logging
import re
import weakref
 
import cv2

import time
import h5py

import PIL.Image
from io import BytesIO
import IPython.display

import h5py  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.0M  100 21.0M    0     0  8648k      0  0:00:02  0:00:02 --:--:-- 8645k


In [3]:
class Sensors(object):
    def __init__(self):
        self.camera = dict()

class Measurement(object):  
    def __init__(self):
        self.steer = None
        self.gas = None
        self.brake = None
        self.velocity = None
        self.rgb = None
        
class World(object):
    
    restarted = False

    def __init__(self, client):
        self.client = client
        self.world = client.get_world()
        self._roles_names = ['ego', 'hero', 'hero1', 'hero2','hero3', 'hero4','hero5', 'hero6']
        self.actor_role_name = random.choice(self._roles_names)
        self._sensors = Sensors()
        self._measurements = dict()
        
        try:
            self.map = self.world.get_map()
        except RuntimeError as error:
            logging.error('RuntimeError: {}'.format(error))
            logging.error('  The server could not send the OpenDRIVE (.xodr) file:')
            logging.error('  Make sure it exists, has the same name of your town, and is correct.')
            sys.exit(1)
        self.player = None
        self.camera_manager = None
        self._actor_filter = 'mustang'
        self._gamma = 2.2
        self.restart()

    def restart(self):
        self.player_max_speed = 1.589
        self.player_max_speed_fast = 3.713
        # Keep same camera config if the camera manager exists.
        cam_index = self.camera_manager.index if self.camera_manager is not None else 0
        cam_pos_index = self.camera_manager.transform_index if self.camera_manager is not None else 0
        # Get a random blueprint.
        blueprint = random.choice(self.world.get_blueprint_library().filter(self._actor_filter))
        blueprint.set_attribute('role_name', self.actor_role_name)
        if blueprint.has_attribute('color'):
            color = random.choice(blueprint.get_attribute('color').recommended_values)
            blueprint.set_attribute('color', color)
        if blueprint.has_attribute('driver_id'):
            driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
            blueprint.set_attribute('driver_id', driver_id)
        if blueprint.has_attribute('is_invincible'):
            blueprint.set_attribute('is_invincible', 'true')
        # set the max speed
        if blueprint.has_attribute('speed'):
            self.player_max_speed = float(blueprint.get_attribute('speed').recommended_values[1])
            self.player_max_speed_fast = float(blueprint.get_attribute('speed').recommended_values[2])
        else:
            print("No recommended values for 'speed' attribute")
        # Spawn the player.
        if self.player is not None:
            spawn_point = self.player.get_transform()
            spawn_point.location.z += 2.0
            spawn_point.rotation.roll = 0.0
            spawn_point.rotation.pitch = 0.0
            self.destroy()
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
        while self.player is None:
            if not self.map.get_spawn_points():
                print('There are no spawn points available in your map/town.')
                print('Please add some Vehicle Spawn Point to your UE4 scene.')
                sys.exit(1)
            spawn_points = self.map.get_spawn_points()
            spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
            
        self.camera_manager = CameraManager(self.player, self._gamma, 384, 160)
            
    def destroy_sensors(self):
        self.camera_manager.sensor.destroy()
        self.camera_manager.sensor = None
        self.camera_manager.index = None

    def destroy(self):
        actors = [
            self.camera_manager.sensor,
            self.player]
        for actor in actors:
            if actor is not None:
                actor.destroy()
    
    def get_measurements(self):
        control = self.player.get_control()
        vel = self.player.get_velocity()
        velocity = math.sqrt(vel.x ** 2 + vel.y ** 2 + vel.z ** 2)
        #img = PIL.Image.fromarray(np.uint8(self.camera_manager.surface)).convert('RGB')
        #return (control.steer, control.throttle, control.brake, velocity, np.asarray((img), dtype="uint8"))
        measurement = Measurement()
        measurement.steer = control.steer
        measurement.gas = control.throttle
        measurement.brake = control.brake
        measurement.velocity = velocity
        measurement.rgb = self.camera_manager.surface
        return measurement

In [4]:
class CameraManager(object):
  
    def __init__(self, parent_actor, gamma_correction, width = 640, height = 480):
        self.width = width
        self.height = height
        self.sensor = None
        self.surface = np.zeros(shape=(self.height, self.width, 3), dtype=np.uint8)
        self._parent = parent_actor
        
        bound_y = 0.5 + self._parent.bounding_box.extent.y
        Attachment = carla.AttachmentType
        self._camera_transforms = [
            (carla.Transform(carla.Location(x=1.6, z=1.7)), Attachment.Rigid),
        ]

        world = self._parent.get_world()
        bp_library = world.get_blueprint_library()
        bp = bp_library.find('sensor.camera.rgb')
        bp.set_attribute('image_size_x', str(self.width))
        bp.set_attribute('image_size_y', str(self.height))
        if bp.has_attribute('gamma'):
            bp.set_attribute('gamma', str(gamma_correction))
        
        self.sensor = self._parent.get_world().spawn_actor(bp,self._camera_transforms[0][0],attach_to=self._parent,attachment_type=self._camera_transforms[0][1])     
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda image: CameraManager._parse_sensor(weak_self, image, 0)) 
    
    @staticmethod
    def _parse_sensor(weak_self, image, sensor_index=0):
        self = weak_self()
        if not self:
            return
        self.surface = CameraManager._parse_image(weak_self, image, sensor_index)
        
    @staticmethod
    def _parse_image(weak_self, image, sensor_index=0):
        self = weak_self()
        if not self:
            return
        image.convert(cc.Raw)
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]
        array = array[:, :, ::-1]
        return array.copy()

In [5]:
log_level = logging.INFO
logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)
world = None

try:
    logging.info('🐧🐧🐧 Start 🐧🐧🐧')
    
    client = carla.Client('127.0.0.1', 2000)
    #client.set_timeout(5.0)
    #client.load_world('Town02')
    
    world = World(client) 
    world.player.set_autopilot(True)
    logging.info('Starting test')
    
    sample_time = 1
    dataset_num = 1
    dataset_count = 0
    batch = 200
    count = 0
   
    dataset_measurements = []
    dataset_rgb = []
    
    SPECIAL_TYPE = np.dtype([
        ("steer", np.float32),
        ("throttle", np.float32),
        ("brake", np.float32),
        ("velocity", np.float32)
    ])
    
    while dataset_count < dataset_num:
        observations = world.get_measurements()
        dataset_measurements.append((observations.steer, observations.gas, observations.brake, observations.velocity))
        dataset_rgb.append(observations.rgb)
        
        if count == batch:
            hdf = h5py.File("carla-dataset-{}.h5".format(dataset_count),'w')
            hdf.create_dataset("measurements".format(dataset_count), data=np.array(dataset_measurements, dtype=SPECIAL_TYPE) , dtype=SPECIAL_TYPE)
            hdf.create_dataset("rgb".format(dataset_count), data=dataset_rgb, dtype='uint8')
            count = 0
            dataset = []
            dataset_count = dataset_count + 1
            
        time.sleep(sample_time)    
        count = count + 1
        
    hdf.close()
    logging.info('🐧🐧🐧 Fin!🐧🐧🐧')
    
except KeyboardInterrupt:
    logging.info('\nCancelled by user. Bye!')
    hdf.close()
    if world is not None:
        world.destroy()

INFO: 🐧🐧🐧 Start 🐧🐧🐧


No recommended values for 'speed' attribute


INFO: Starting test
INFO: 
Fin!
